<a href="https://colab.research.google.com/github/MaengJoonYoung/DL-Project/blob/main/car_damaged.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import EfficientNetB0, MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization, Dropout
import IPython
import os

In [ ]:
!pip install -U keras-tuner
import kerastuner as kt

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# seed 설정
tf.random.set_seed(42)
np.random.seed(42)

In [6]:
IMG_SIZE = 224
BATCH_SIZE = 32

In [7]:
# 데이터 경로
data_path = '/content/drive/MyDrive/car_damage/'
train_dir = os.path.join(data_path, 'training/')
test_dir = os.path.join(data_path, 'validation/')

In [7]:
#데이터셋 확인

num_train_damage = len(os.listdir(train_dir + '00-damage/'))
num_train_whole = len(os.listdir(train_dir + '01-whole/'))
tot_train = num_train_damage + num_train_whole

num_test_damage = len(os.listdir(test_dir + '00-damage/'))
num_test_whole = len(os.listdir(test_dir + '01-whole/'))
tot_test = num_test_damage + num_test_whole

print(f"""
훈련 데이터셋 파손 데이터 수 : {num_train_damage}
훈련 데이터셋 정상 데이터 수 : {num_train_whole}
훈련 데이터셋 총  데이터 수 : {tot_train}
----------------------------------
검증 데이터셋 파손 데이터 수 : {num_test_damage}
검증 데이터셋 정상 데이터 수 : {num_test_whole}
검증 데이터셋 총  데이터 수 : {tot_test}
""")


훈련 데이터셋 파손 데이터 수 : 920
훈련 데이터셋 정상 데이터 수 : 920
훈련 데이터셋 총  데이터 수 : 1840
----------------------------------
검증 데이터셋 파손 데이터 수 : 230
검증 데이터셋 정상 데이터 수 : 230
검증 데이터셋 총  데이터 수 : 460



In [8]:
# generator 생성

# 이미지 증강
train_gen = image.ImageDataGenerator(width_shift_range=0.2,
                                     brightness_range = (0.2, 0.1),
                                     horizontal_flip = True,
                                     rescale= 1./255,
                                     fill_mode='nearest')

test_gen = image.ImageDataGenerator(rescale= 1./255)

In [9]:
# 데이터 불러오기

train = train_gen.flow_from_directory(train_dir,
                                      target_size=(IMG_SIZE,IMG_SIZE),
                                      batch_size=BATCH_SIZE,
                                      color_mode='rgb',
                                      shuffle=True,
                                      class_mode='binary')

test = test_gen.flow_from_directory(test_dir,
                                      target_size=(IMG_SIZE,IMG_SIZE),
                                      batch_size=BATCH_SIZE,
                                      color_mode = 'rgb',
                                      shuffle=True,
                                      class_mode='binary')

Found 1840 images belonging to 2 classes.
Found 460 images belonging to 2 classes.


In [10]:
print('train_classes', train.class_indices)
print('test_classes', test.class_indices)

train_classes {'00-damage': 0, '01-whole': 1}
test_classes {'00-damage': 0, '01-whole': 1}


In [ ]:
# 사전학습 모델 불러오기.

base_model = MobileNetV2(weights='imagenet', input_shape=(224,224,3),include_top=False)
base_model.trainable = False

In [ ]:
# 모델 구성

model_1 = Sequential()

model_1.add(base_model)
model_1.add(GlobalAveragePooling2D())
model_1.add(Dense(1024, activation='relu'))
model_1.add(BatchNormalization())
model_1.add(Dense(512, activation='relu'))
model_1.add(BatchNormalization())
model_1.add(Dense(1, activation='sigmoid'))

model_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [ ]:
history = model_1.fit(train, epochs=3, batch_size=32 ,validation_data=test)

Epoch 1/3
58/58 [==============================] - 447s 8s/step - loss: 0.5539 - acc: 0.8152 - val_loss: 0.7728 - val_acc: 0.6674
Epoch 2/3
58/58 [==============================] - 41s 712ms/step - loss: 0.3321 - acc: 0.8620 - val_loss: 0.8282 - val_acc: 0.7022
Epoch 3/3
58/58 [==============================] - 41s 710ms/step - loss: 0.2540 - acc: 0.8908 - val_loss: 0.4055 - val_acc: 0.8217


In [58]:
# 하이퍼파라미터 조절을 위해 model_builder 함수 정의.
def model_builder(hp):

  base = MobileNetV2(weights='imagenet', input_shape=(IMG_SIZE,IMG_SIZE,3),include_top=False)
  for layer in base.layers[:-4]:
    layer.trainable=False

  model_tuned = Sequential()

  model_tuned.add(base)
  model_tuned.add(GlobalAveragePooling2D())

  hp_units = hp.Int('units', min_value=32, max_value=1024, step=32)
  model_tuned.add(Dense(units=hp_units, kernel_initializer='he_normal', activation='relu'))

  hp_dropout = hp.Float('dropout', min_value=0.1, max_value=0.5, step=0.1)
  model_tuned.add(Dropout(hp_dropout))

  hp_units_2 = hp.Int('units_2', min_value=32, max_value=1024, step=32)
  model_tuned.add(Dense(units=hp_units_2, kernel_initializer='he_normal',activation='relu'))
  model_tuned.add(Dropout(hp_dropout))
  model_tuned.add(Dense(1, activation='sigmoid'))

  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model_tuned.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=hp_learning_rate), loss='binary_crossentropy', metrics=['acc'])

  return model_tuned

In [59]:
# BayesianOptimization
tuner = kt.BayesianOptimization(model_builder,
                                objective='val_acc',
                                max_trials=10,
                                num_initial_points=2,
                                seed=42)

In [60]:
# trial 후 사라지게.
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

In [61]:
tuner.search(train, epochs=7, validation_data=test, callbacks=[ClearTrainingOutput()])

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
최적의 첫번재 노드 수 : {best_hps.get('units')}
최적의 두번째 노드 수 : {best_hps.get('units_2')}
최적의 학습률 : {best_hps.get('learning_rate')}
최적의 dropout : {best_hps.get('dropout')}
""")

Trial 10 Complete [00h 04m 59s]
val_acc: 0.834782600402832

Best val_acc So Far: 0.873913049697876
Total elapsed time: 00h 54m 27s
INFO:tensorflow:Oracle triggered exit

최적의 첫번재 노드 수 : 928
최적의 두번째 노드 수 : 576
최적의 학습률 : 0.01
최적의 dropout : 0.2



In [10]:
# colab 연결 끊겨서 수동으로 파라미터 적음.

base = MobileNetV2(weights='imagenet', input_shape=(IMG_SIZE,IMG_SIZE,3),include_top=False)
for layer in base.layers[:-4]:
  layer.trainable = False

model_tuned = Sequential()

model_tuned.add(base)
model_tuned.add(GlobalAveragePooling2D())
model_tuned.add(Dense(928, kernel_initializer='he_normal', activation='relu'))
model_tuned.add(Dropout(0.2))
model_tuned.add(Dense(576, kernel_initializer='he_normal', activation='relu'))
model_tuned.add(Dropout(0.2))
model_tuned.add(Dense(1, activation='sigmoid'))

model_tuned.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.01), loss='binary_crossentropy', metrics=['acc'])

9420800/9406464 [==============================] - 0s 0us/step


In [11]:
# early_stop 적용 (val_loss가 제일 낮은 epoch 저장.)
checkpoint = 'FMbest.hdf5'

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=1)

save_best = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint, monitor='val_loss', verbose=1, save_best_only=True,
                                               save_weights_only=True, mode='auto', save_freq='epoch', options=None)

In [13]:
model_tuned.fit(train,batch_size=32, epochs=50, verbose=1,
          validation_data=test,
          callbacks=[early_stop,save_best])

Epoch 1/50
58/58 [==============================] - ETA: 0s - loss: 0.4878 - acc: 0.7587
Epoch 00001: val_loss improved from inf to 0.53986, saving model to FMbest.hdf5
58/58 [==============================] - 646s 11s/step - loss: 0.4878 - acc: 0.7587 - val_loss: 0.5399 - val_acc: 0.7196
Epoch 2/50
58/58 [==============================] - ETA: 0s - loss: 0.3931 - acc: 0.8250
Epoch 00002: val_loss did not improve from 0.53986
58/58 [==============================] - 41s 704ms/step - loss: 0.3931 - acc: 0.8250 - val_loss: 0.6613 - val_acc: 0.6500
Epoch 3/50
58/58 [==============================] - ETA: 0s - loss: 0.3571 - acc: 0.8516
Epoch 00003: val_loss improved from 0.53986 to 0.47847, saving model to FMbest.hdf5
58/58 [==============================] - 41s 708ms/step - loss: 0.3571 - acc: 0.8516 - val_loss: 0.4785 - val_acc: 0.7587
Epoch 4/50
58/58 [==============================] - ETA: 0s - loss: 0.3289 - acc: 0.8598
Epoch 00004: val_loss improved from 0.47847 to 0.46121, saving m

In [15]:
# best 모델 불러오기.
model_tuned.load_weights(checkpoint)